In [1]:
import cv2
import os
import numpy as np
from pathlib import Path

In [ ]:
'''
El código toma las carpetas de mascaras y segmentaciones realizadas (se deben
especificar al final del código en la funcion main) luego de esto guarda las imagenes resultantes
en las carpetas destino de segmentacion aumentada y mask aumentada (también se deben
especificar en la funcion main)

Las imagenes resultantes por cada segmentación y por cada máscara son
- reflexion vertical (en caso de las segmentadas también cambia el contraste)
- reflexion horizontal (en caso de las segmentadas también cambia el contraste)
- reflexion vertical y horizontal (en caso de las segmentadas también cambia el contraste)
- imagen original 

Por lo que cada imagen tendrá 4 imágenes resultantes en las carpetas destino
'''

import cv2
import os
import numpy as np
from pathlib import Path

def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def apply_contrast_multiplication(image, factor):
    return np.clip(image * factor, 0, 255).astype(np.uint8)

def process_images(source_segmented, source_masks, output_segmented, output_masks):
    # Create output directories
    create_directory(output_segmented)
    create_directory(output_masks)
    
    # Generate contrast factors (excluding 1.0)
    contrast_factors = np.concatenate([
        np.linspace(0.80 , 0.90, 10),
        np.linspace(1.10 , 1.20, 10)
    ])
    
    # Process each pair of images
    for img_path in Path(source_segmented).glob('*.png'):  # Adjust extension if needed
        # Get corresponding mask name
        base_name = img_path.stem
        mask_path = Path(source_masks) / f"{base_name.replace('_segmented', '_mask')}.png"
        
        if not mask_path.exists():
            print(f"Warning: No corresponding mask found for {img_path}")
            continue
            
        # Read images
        img_segmented = cv2.imread(str(img_path))
        img_mask = cv2.imread(str(mask_path))
        
        if img_segmented is None or img_mask is None:
            print(f"Error reading images for {base_name}")
            continue
            
        # Save original images
        cv2.imwrite(os.path.join(output_segmented, f"{base_name}_original.png"), img_segmented)
        cv2.imwrite(os.path.join(output_masks, f"{base_name.replace('_segmented', '_mask')}_original.png"), img_mask)
        
        # Create reflections with random contrast for segmented images
        reflections = {
            'vertical': cv2.flip(img_segmented, 0),
            'horizontal': cv2.flip(img_segmented, 1),
            'hor_ver': cv2.flip(cv2.flip(img_segmented, 1), 0)
        }
        
        # Create reflections for masks (without contrast adjustment)
        mask_reflections = {
            'vertical': cv2.flip(img_mask, 0),
            'horizontal': cv2.flip(img_mask, 1),
            'hor_ver': cv2.flip(cv2.flip(img_mask, 1), 0)
        }
        
        # Save all variations
        for reflection_type, reflected_img in reflections.items():
            # Random contrast factor
            contrast_factor = np.random.choice(contrast_factors)
            
            # Apply contrast to segmented image and save
            contrasted_img = apply_contrast_multiplication(reflected_img, contrast_factor)
            output_name = f"{base_name}_{reflection_type}_reflection_contraste.png"
            cv2.imwrite(os.path.join(output_segmented, output_name), contrasted_img)
            
            # Save mask reflection (without contrast)
            mask_output_name = f"{base_name.replace('_segmented', '_mask')}_{reflection_type}_reflection.png"
            cv2.imwrite(os.path.join(output_masks, mask_output_name), mask_reflections[reflection_type])

def main(source_segmented, source_masks):
    # Define directories
    output_segmented = source_segmented + "_aumentado" #Carpeta del dataset aumentado de segmentados destino
    output_masks = source_masks + "_aumentado" #Carpeta del dataset aumentado de mascaras destino
    
    # Process the images
    process_images(source_segmented, source_masks, output_segmented, output_masks)
    print("Augmentation completed successfully!")

main("Ulcers_segmented","Ulcers_mask")

Augmentation completed successfully!
